# Field-level NDVI Time Series
In this notebook we will use Descartes Labs `Workflows` to build an interactive map to **query the time series of vegetative health for any field in the world**.  The field boundary is drawn by the user, and the NDVI time series from Sentinel-2 is returned.  This allows the user to assess **field-level vegetative health** or determine **crop planting patterns such as winter cover crops** in an interactive, visual way.

You can run the following cells using `Shift-Enter`.

## Import packages

In [1]:
# keep logging quiet
import logging

logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [2]:
# import packages
import descarteslabs.workflows as wf

#Integrating multiple flows means multiple utils
from field_ndvi_utils import FieldNDVI
import utils

In [3]:
# import dl_weather

# # For generating the geocontext:
# from IPython import display as dsp
# import shapely.geometry as sg
# import descarteslabs as dl
# import geojson
# import numpy as np

# # For plotting timeseries:
# import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'dl_weather'

## Create a weather analysis geocontext - in this case, in California's Central Valley

In [4]:
# wf.map.center = [ 38.7752, -121.7608 ] # This is my center of map
# wf.map.zoom = 12

# wf.map



In [5]:
# bounds = np.array(wf.map.bounds).flatten()
# bounds = [bounds[1], bounds[0], bounds[3], bounds[2]]
# utm_code = utils.wgs_to_epsg(*wf.map.center)


# ctx = wf.GeoContext(
#     bounds=bounds,
#     bounds_crs="EPSG:4326",
#     crs=f"EPSG:{utm_code}",
#     resolution = 10.
# )

In [6]:
# %%time
# ts = dl_weather.get_weather_time_series(weather_product="ecmwf:era5:v0",
#                                         weather_vars = ["tp",'100u'],
#                                         weather_start = '2018-01-01',
#                                         weather_end = '2018-02-01',
#                                         frequency='daily',
#                                         geoctx=ctx)
# ts.head()

In [7]:
# plt.figure(figsize=(8,6))
# plt.subplot(2,1,1)
# ts['tp'].plot(color='blue', label='Precipitation')
# plt.grid('lightgray')
# plt.legend()
# plt.subplot(2,1,2) 
# ts['100u'].plot(color='orange', label='Zonal wind speed')
# plt.grid('lightgray')
# plt.legend()
# plt.tight_layout()


## Define the _Sentinel-2_ imagery that will display on the map

In [8]:
s2 = wf.ImageCollection.from_id(
    "sentinel-2:L1C", start_datetime="2021-03-01", end_datetime="2021-07-01"
)

s2 = s2.filter(lambda img: img.properties["cloud_fraction"] <= 0.05)
s2 = s2.pick_bands("nir red green").median(axis="images")
s2.visualize(
    "Sentinel- NRG", scales=[[0, 0.3], [0, 0.3], [0, 0.3]], checkerboard=False
)

And the USDA Cropland Data Layer


In [9]:
cdl = wf.ImageCollection.from_id(
    "usda:cdl:v1", start_datetime="2020-12-31", end_datetime="2021-01-01"
)


cdl.mask(cdl != 54).visualize('CDL Tomatoes',
                             colormap='spring')
cdl_mask = cdl.mosaic() != 54

In [10]:
s2_new = s2.mask(cdl_mask).visualize(
    "Sentinel-2 mmasked", scales=[[0, 0.3], [0, 0.3], [0, 0.3]], checkerboard=False
)

## Create the `FieldNDVI` object, a custom widget defined in `field_ndvi.py`
This is the widget that will compute the NDVI time series of your field of interest.  The date range for this time series takes on a default value but can be adjusted once you make the map below.

In [11]:
ndvi_timeseries = FieldNDVI(wf.map)

## Define the map center and zoom level.
We choose a field in Iowa that appears to have a **winter cover crop** in early 2018 but not in early 2019.

In [12]:
wf.map.center =  36.334871, -120.227047  # Iowa
wf.map.zoom = 12



## Finally, display and interact with the map.
Select the polygon draw tool on the left and draw a polygon over your field of interest.  This will trigger a comptutation on the Descartes Labs `Workflows` backend to compute the mean NDVI in this field, which will then display embedded in the map.

In [13]:
wf.map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


### If desired, return the NDVI time series as a pandas `dataframe`.
You must first draw an AOI in the map above.  Then uncomment and run the line below.

In [ ]:
ndvi_timeseries.df

In [ ]:
ndvi_timeseries.geoctx.bounds